In [1]:
import zipfile
from urllib.request import urlopen
import io
from pandas import read_csv, DataFrame, Series
from pandas.errors import PerformanceWarning
from math import log as ln
from statistics import mean
from seaborn import violinplot

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=PerformanceWarning)
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff

# from googletrans import Translator, constants
# translator = Translator()
# def tr(s, thru_en = False):
#   if thru_en:
#     ss = translator.translate(s, src = 'nl', dest = 'en').text
#     return translator.translate(ss, src = 'en', dest = 'ru').text
#   else:
#     return translator.translate(s, src = 'nl', dest = 'ru').text
# def tr_en(s):
#     return translator.translate(s, src = 'nl', dest = 'ru').text

In [2]:
path = "https://lodmedia.hb.bizmrg.com/case_files/1125024/train_dataset_vprod_train.zip"

z = zipfile.ZipFile(io.BytesIO(urlopen(path).read()))


In [3]:
ts=z.open("vprod_train/TRAIN_SAL.csv")
prom = read_csv(ts)
df_ts = prom.head(200000)
# df_provinces['date'] = df_provinces['date'].map(lambda x: x[:7])


del z, prom

for i in df_ts.columns:
  if len(df_ts[i].unique())==1:
    df_ts = df_ts.drop(i, axis=1)


<ipython-input-3-1ed2eef8202c>:2: DtypeWarning: Columns (1,16,24,69) have mixed types. Specify dtype option on import or set low_memory=False.
  prom = read_csv(ts)


In [4]:
import zipfile
from urllib.request import urlopen
import io
from pandas import read_csv

# Ввод данных

In [ ]:
# import pandas as pd
# df_ts = pd.read_csv('TRAIN_SAL.csv')

# Обработка

0. В общих чертах

- Вручную посмотрели на количество уникальных значений, формат непустых значений признаков
- Поняли, что имеем в сырых данных:

а) числа

б) категории

в) связный текст

г) текст с JSON

д) текст с набором "меток" (в одном признаке) — разбили на отдельные категории true-false

е) константы — от них избавились

ж) Текст внутри разметки, похожей на HTML — не зная глубоко, посчитали сересчур трудоёмким придумывать, как искать особенные случаи по 600 тыс. записям, и токенизировали прямо с разметкой (могли вырезать по "<" и ">", могли добавить стоп-слова)

з) дату-время — не придумали, как обосновать выбор формата для них для подачи в модели, потому не включили из в "первый прогон" (и посчитали, что первый прогон сработал достаточно хорошо, чтобы переключиться на другие задачи)

- Текст подали на TfIdfVectorizer — для первой попытки "попробовать бустинг"
- Категории: в итоге смогли поднять бустинг из scikitlearn. В документации сказано, что в нём категориальные признаки в сыром виде не принимаются, принимаются числа. Есть официальная статья, в которой предлагаются 4 способа преобразования в числа. Выбрали один из них: прогнать категории через OrdinalEncoder

1. В фичах есть такие, в которых для всех записей одно и то же значение. На модель в лучшую сторону они однозначно не повлияют

In [5]:
# убираем фичи, для которых количесвто уникальных значений == 1
for i in df_ts.columns:
  if len(df_ts[i].unique())==1:
    df_ts = df_ts.drop(i, axis=1)


2. Категории VS Числа VS Тексты

Для подачи ML-ю на вход категориальные / числовые / языковые данные преобразовываются кардинально по-разному.

Казалось лёгким применить функции из коробки, чтобы не вручную перебирать все признаки. Но потом пришлось, т.к. автоматическое выявление на этих данных не отработало.

Поэтому сначала будет выделено 3  листа под 3 типа признаков, а затем несколько раз названия колонок будут перебрасываться из одного листа в другой (в тех моментах, в которые мы, исторически, ловили разные ошибки, пытаясь запихнуть в обработку неверные данные второпях).

2.1. Автоматически выбираем тексты и числа

In [6]:
str_columns = df_ts.applymap(lambda x: isinstance(x, str)).all(0)

text_features = []
for i in list(str_columns.index):
  if str_columns[i]:
    text_features.append(i)
text_features.remove("id")


In [7]:
nums = df_ts._get_numeric_data().columns
nums = list(nums)


2.2. Посмотрели по каждой фиче: количество уникальных, если мало — все значения, если много — первые 100 уникальных, чтобы понять формат.

Поняли, что не знаем, как тривиально обрабатывать datetime — решили "пока без него" (и не вернулись к нему. "Если мы это забыли, значит это не так уж и важно", — или, формальнее: "Нас удовлетворил итоговый скор".)

In [8]:
datetime_features = ["change_time", ]

Поняли, что есть гнусные колонки, содержащие неатомарные данные. multicats включает в себя наборы категориальных меток. Здесь и далее каждая метка переводится в отдельный признак с двумя-тремя значениями (в зависимости от того, есть ли возможность указывать там nan, который мы по максимуму оставляли).

In [9]:
multicats = ["social_protected_ids", "vacancy_benefit_ids"]

2.3. Разбили неатомарные данные (пока только те, которые нашли в автоматически определённых к строковым, и по малому количеству уникальных строк)

In [10]:
social_protected_ids = ["Беженцы",
                        "лица, получившие временное убежище",
                        "Инвалиды",
                        "Лица, освобождаемые из мест лишения свободы",
                        "Матери и отцы, воспитывающие без супруга (супруги) детей в возрасте до пяти лет",
                        "вынужденные переселенцы",
                        "Многодетные семьи",
                        "Несовершеннолетние работники",
                        "Работники, имеющие детей-инвалидов",
                        "Работники, осуществляющие уход за больными членами их семей в соответствии с медицинским заключением",
                        # "nan"
                        ]

In [11]:
vacancy_benefit_ids = [
    "ДМС",
    "Путевки в оздоровительные учреждения",
    "Оплата занятий спортом",
    "Оплата питания",
]

Код для "вычёркивания из уже учтённых" при выборе меток руками (выбор = записали в листы выше; перезапустили и перепроверили, что кроме пустоты текста не осталось)

In [ ]:
# @title
spi_raw = sorted([str(i) for i in list(df_ts["social_protected_ids"].unique())])
for i in social_protected_ids:
  for j in range(len(spi_raw)):
    if i in spi_raw[j]:
      spi_raw[j] = spi_raw[j].replace(i, "")
spi_raw

['',
 '; ; ',
 '; ; ,',
 '; ; ,,',
 '; ; ,,,',
 '; ; ,,,,',
 '; ; ,,,,,',
 '; ; ,,,,,,',
 '; ; ,,,,,,,',
 '; ; ,,,,,',
 '; ; ,,,,,,',
 '; ; ,,,,,',
 '; ; ,,,,,',
 '; ; ,,,',
 '; ; ,,,,',
 '; ; ,,,,,',
 '; ; ,,,,',
 '; ; ,,,,,',
 '; ; ,,,',
 '; ; ,,,',
 '; ; ,,,,',
 '; ; ,,',
 '; ; ,,,',
 '; ; ,,,,',
 '; ; ,,,,,',
 '; ; ,,,,,,',
 '; ; ,,,,',
 '; ; ,,,,,',
 '; ; ,,,,',
 '; ; ,,,',
 '; ; ,,,,',
 '; ; ,,,',
 '; ; ,,',
 '; ; ,,,',
 '; ; ,,,,,',
 '; ; ,,,',
 '; ; ,,,,',
 '; ; ,,,',
 '; ; ,,',
 '; ; ,,',
 '; ; ,,',
 '; ; ,',
 '; ; ,,',
 '; ; ,,,',
 '; ; ,,,,',
 '; ; ,,,,,',
 '; ; ,,,,,,',
 '; ; ,,,,',
 '; ; ,,,,,',
 '; ; ,,,,',
 '; ; ,,,,',
 '; ; ,,,',
 '; ; ,,',
 '; ; ,,,',
 '; ; ,,,',
 '; ; ,,,,',
 '; ; ,,',
 '; ; ,,',
 '; ; ,',
 '; ; ,,',
 '; ; ,,,',
 '; ; ,,,,',
 '; ; ,,,,,',
 '; ; ,,,',
 '; ; ,,,,',
 '; ; ,,,',
 '; ; ,,,,',
 '; ; ,,',
 '; ; ,,,',
 '; ; ,',
 '; ; ,,',
 '; ; ,,,',
 '; ; ,,',
 '; ; ,,,',
 '; ; ,',
 '; ; ,',
 '; ; ,,',
 '',
 ',',
 ',,',
 ',,,',
 ',,,,,',
 ',,,,,,',
 ',,,,,',

In [ ]:
# @title
vbi_raw = sorted([str(i) for i in list(df_ts["vacancy_benefit_ids"].unique())])
for i in vacancy_benefit_ids:
  for j in range(len(vbi_raw)):
    if i in vbi_raw[j]:
      vbi_raw[j] = vbi_raw[j].replace(i, "")
vbi_raw

['nan',
 '',
 ',',
 ',,',
 '',
 ',',
 ',',
 ',,',
 ',',
 ',,',
 ',,',
 ',,,',
 '',
 ',',
 '',
 ',',
 ',',
 ',,']

Переложили ошибочные текстовые в нужные типы.
За одно заметили, что id это признак (и обвинили его в ненужности).

In [12]:
cats = []
for i in df_ts.columns:
  if not i in nums and not i in text_features:
    cats.append(i)
try:
  cats.remove("id")
except:
  pass


Сюда выписывали не текстовые но категориальные признаки при просмотре текстовых

In [13]:
false_cats = ["additional_requirements", "contact_person", "education_speciality",
              "metro_ids", "other_vacancy_benefit", "position_requirements",
              "position_responsibilities", "required_certificates",
              "vacancy_address_additional_info", "vacancy_address", "full_company_name" ]

Опять перебрасываем нужное, удаляем переброшенное или не нужное

(try-catch на случай перезапуска при отладке, т.к. при повторном удалении поднимает ошибку)

In [14]:
for i in false_cats:
  try:
    cats.remove(i)
  except:
    pass
  try:
    text_features.append(i)
  except:
    pass

for i in datetime_features:
  try:
    cats.remove(i)
  except:
    pass

In [15]:
for i in datetime_features:
  try:
    cats.remove(i)
  except:
    pass

for i in multicats:
  try:
    cats.remove(i)
  except:
    pass

Снова разбили неатомарные данные на колонки-категории true-false

In [16]:
for feature in social_protected_ids:
  df_ts[feature] = feature in df_ts["social_protected_ids"]
  cats.append(feature)
for feature in vacancy_benefit_ids:
  df_ts[feature] = feature in df_ts["vacancy_benefit_ids"]
  cats.append(feature)

Снова руками перебрали фичи через количество уникальных и по паре (сотен) примеров — для признаков, автоматически определённых к текстовым (т.к. в ходе поднятия моделей нашли ошибки в векторизации).

In [17]:
false_text_actualy_cats = [
    "busy_type",
    "education",
    "original_source_type",
    "company_business_size",
    "retraining_grant",
    "schedule_type",
    "source_type",
    "status",
              ]
false_text_actualy_nums = [
    "company_code",
]

junk = [
    "data_ids",
]

datetime_features += [
    "date_create",
    "date_modify",
    "published_date",
    ]

multicats += ["required_drive_license"]

В требовании водительских прав получили список категорий

In [18]:
driver_cats_interm = df_ts["required_drive_license"].loc[df_ts["required_drive_license"].apply(lambda x: x!='[]')]
driver_cats = []

for i in driver_cats_interm:
  i = i.replace("[","")
  i = i.replace("]","")
  i = i.replace("'", "")
  i = i.replace('"', "")
  i = i.split(",")
  for k in i:
    if not k in driver_cats:
      driver_cats.append(k)


...и породили новые колонки в датафрейме

In [19]:
for cat in driver_cats:
  df_ts[f"driver_cat_{cat}"] = df_ts["required_drive_license"].apply(lambda x: "True" if cat in x else "False")
  cats.append(f"driver_cat_{cat}")

try:
  text_features.remove("required_drive_license")
except:
  pass


..., удалив из датафрейма исходный комбинированный признак, чтобы влезло больше данных по памяти

In [20]:
df_ts = df_ts.drop("required_drive_license", axis=1)

И снова перебросили

In [21]:
for i in false_text_actualy_cats:
  try:
    cats.append(i)
  except:
    pass
  try:
    text_features.remove(i)
  except:
    pass
for i in false_text_actualy_nums:
  try:
    nums.append(i)
  except:
    pass
  try:
    text_features.remove(i)
  except:
    pass
for i in junk:
  try:
    text_features.remove(i)
  except:
    pass
for i in datetime_features:
  try:
    text_features.remove(i)
  except:
    pass

for i in multicats:
  try:
    text_features.remove(i)
  except:
    pass

3. (Вздохнули и) Раскрыли JSON признаки

In [22]:
"company", "languageKnowledge", "hardSkills", "softSkills" # json xD # body count = 2 (dead)

('company', 'languageKnowledge', 'hardSkills', 'softSkills')

In [23]:
from json import loads

Проверили, какие ключи есть в JSOn-ах. Проверили на одном примере для быстроты, что они полностью повторяются. При запуске обработке, опирающейся на это, вылезла ошибка, поэтому в итоге учли то, что они всё же не повторяются полностью.

В JSON много данных, в сущности определяющих работодателя. Дабы не дублировать, оставили одно поле. В этом поле (хотя оно в большинстве случаев дулирует до этого существующую колонку в датафрейме) есть строки, в которых из JSONа строка отличается от строки из другой колонки датафрейма. Решили оставить name, чтобы в таких случаях можно было подобрать дополнительные токены из текста, посудив о зарплате по встрече этих токенов в контексте резюме. Альтернативный вариант — целочисленный ID работодателя — такой возможности не даёт, возможность учитывать его как категорию при малом количестве вакансий к одному работодателю и появлении новых интуитивно показаласт менее выигрышной, представлять его как числовой признак показалось плохо.

In [24]:
keys_comp = []
for i in list(df_ts["company"]):
  for k in loads(i):
    if not k in keys_comp:
      keys_comp.append(k)

for k in keys_comp:
  if not k in loads(str(df_ts["company"].head(1).iloc[0])):
    print(k) #it's ok if it prints nothing, ergo (suppose...) all the keys always repeat
    # aaaaand it's not true in general

''' # if we desperately need ALL THE DATA
for k in keys_comp:
  df_ts[f"company_{k}"] = df_ts['company'].apply(lambda x: loads(x)[k])

# print(keys_comp) #(for those keys_comp TODO:..)
#TODO cats.append()
#TODO nums.append()
#TODO text_features.append()
'''

df_ts["company_json_name"] = df_ts["company"].apply(lambda x: loads(x)["name"]
                                                    if "name" in loads(x)
                                                    else "Название не указано") #
'''
наверное найдутся корреляции между совпадениями в названиях и уровнем зарплаты.
А остальное "Если забыл, значит это не так уж и важно" (хочу забыть).
Есть вариант брать по коду компании, чтобы не гонять в тект, затем в числа,
но вдруг они не будут повторяться от слова совсем, а какие-нибудь
"ГБУЗ Поликлиника" будут повторяться для схожих ЗП
'''
text_features.append("company_json_name")


В знании языка есть сам язык и уровень владения. Уникальных значений было не так много, но мы решили сделать вывод, что уровень владения от резюме к резюме жёстко повторяется из жёсткого набора (т.е. оба извлекаемых признака категориальные).

In [25]:
keys_lg = []
for i in list(df_ts["languageKnowledge"]):
  if i!="[]":
    for k in loads(i)[0]:
      if not k in keys_lg:
        keys_lg.append(k)

for k in keys_lg:
  if not k in loads(df_ts["languageKnowledge"].loc[df_ts["languageKnowledge"]
      .apply(lambda x: x!='[]')].head(1).iloc[0])[0]:
    print(k) #it's ok if it prints nothing, ergo (suppose...) all the keys always repeat

In [26]:
df_ts["LK_code_language"] = df_ts["languageKnowledge"].apply(lambda x: loads(x)[0]["code_language"] if x!="[]" else "nan")

In [27]:
df_ts["LK_level"] = df_ts["languageKnowledge"].apply(lambda x: (loads(x)[0]["level"] if "level" in loads(x)[0] else "Не указан") if x!="[]" else "nan")

cats.append("LK_code_language")
cats.append("LK_level")

In [28]:
cats.remove("LK_code_language")
cats.remove("LK_level")
nums.remove("retraining_grant_value")

In [29]:
nums.remove("salary_min")
nums.remove("salary_max")

Хард и софт скилы в JSON-ах содержат только одно осмысленное значение — сам скил

In [30]:
df_ts["hardSkills"] = df_ts["hardSkills"].apply(lambda x: loads(x)[0]["hard_skill_name"] if x!="[]" else "nan")

In [31]:
df_ts["softSkills"] = df_ts["softSkills"].apply(lambda x: loads(x)[0]["soft_skill_name"] if x!="[]" else "nan")

Удаляем лишнее исходное

In [32]:
df_ts = df_ts.drop("company",axis=1)

In [33]:
df_ts = df_ts.drop("languageKnowledge",axis=1)

In [34]:
try:
  text_features.remove("company")
  text_features.remove("languageKnowledge")
except:
  pass

4. Поняли через ошибки, что пропустили числовые признаки, в которых мало значений, и которые могут быть категориями. (И на этом полностью руками перебрали все признаки)

In [35]:
for feature in nums:
  print(feature, end = ": ")
  print(len(list(df_ts[feature].unique())))

accommodation_capability: 2
additional_premium: 531
career_perspective: 2
code_profession: 3419
is_uzbekistan_recruitment: 2
is_quoted: 2
okso_code: 224
publication_period: 2
required_experience: 20
salary: 12041
state_region_code: 90
vacancy_address_code: 39526
vacancy_address_latitude: 70973
vacancy_address_longitude: 71272
work_places: 232
federalDistrictCode: 10
company_inn: 49927
company_code: 52117


In [36]:
junk = ["publication_period", "state_region_code", "company_code"]

In [37]:
false_nums = ["accommodation_capability", "career_perspective",
         "is_uzbekistan_recruitment", "is_quoted", "publication_period",
         "federalDistrictCode"
         ]

cats += false_nums
for i in false_nums:
  try:
    nums.remove(i)
  except:
    pass

try:
  nums.remove("publication_period")
except:
  pass
try:
  nums.remove("state_region_code")
except:
  pass
try:
  nums.remove("company_code")
except:
  pass

In [38]:
list(df_ts["is_quoted"].unique())

[False, True]

Результат - датафрейм с 3 - мя листами, которые идут на вход (cats,nums,test_features) - в каждом листе название обработанных и в будущем использованных столбцов

**target фича - Salary**

Политика обработки nan-ов — оставлять всё, что есть, из данных (nan-ы тоже могут быть данными)


Далее также перевели все категории в строки (дабы погасить ошибку метода преобразования категорий к числам)

На последок проверили, что не потеряли признаки, в случае если будем пользоваться тремя списками колонок

In [39]:
for i in list(df_ts.columns):
  if not i in cats+text_features+nums:
    print(i)

print("------------")

for i in cats+text_features+nums:
  if not i in list(df_ts.columns):
    print(i)

id
change_time
company_code
data_ids
date_create
date_modify
published_date
retraining_grant_value
salary_min
salary_max
social_protected_ids
state_region_code
vacancy_benefit_ids
LK_code_language
LK_level
------------


Избавились от того, от чего хотели избавиться, освободив память

In [41]:
df_ts = df_ts.drop(junk+datetime_features, axis=1)

# Проба моделей

###data preparation

In [43]:
df_ts.columns

Index(['id', 'academic_degree', 'accommodation_capability',
       'accommodation_type', 'additional_premium', 'additional_requirements',
       'bonus_type', 'measure_type', 'busy_type', 'career_perspective',
       'code_external_system', 'code_profession', 'code_professional_sphere',
       'contact_person', 'contact_source', 'data_ids', 'education',
       'education_speciality', 'metro_ids', 'is_mobility_program',
       'is_uzbekistan_recruitment', 'is_quoted', 'need_medcard', 'okso_code',
       'original_source_type', 'other_vacancy_benefit',
       'position_requirements', 'position_responsibilities', 'regionName',
       'company_business_size', 'required_certificates', 'required_experience',
       'retraining_capability', 'retraining_grant', 'retraining_grant_value',
       'salary', 'salary_min', 'salary_max', 'schedule_type',
       'social_protected_ids', 'source_type', 'status',
       'transport_compensation', 'vacancy_address_additional_info',
       'vacancy_address'

Перевод nan в строку

In [44]:
cats.remove("publication_period")

In [58]:
nums.remove("salary")

In [53]:
for i in cats:
  df_ts[i] = df_ts[i].apply(lambda x: str(x))

In [54]:
for feature in cats:
  df_ts[feature] = df_ts[feature].apply(lambda x: "nan" if x!=x else x)# nan всегда != nan

Проверка, что в текстах не осталось конкретного мусора и категорий (ловили ошибку)

In [46]:
for feature in text_features:
  if False in (list(df_ts[feature])):
    print(f"{feature}: \n{df_ts[feature]}\n\n")

In [47]:
df_ts.head(5)

,id,academic_degree,accommodation_capability,accommodation_type,additional_premium,additional_requirements,bonus_type,measure_type,busy_type,career_perspective,...,Оплата занятий спортом,Оплата питания,driver_cat_B,driver_cat_E,driver_cat_C,driver_cat_D,driver_cat_A,company_json_name,LK_code_language,LK_level
0,4e2f52d2-fd76-11e4-8a5b-3bdbd1a6e39d,nan,False,nan,NaN,"<p>Доплаты молодым специалистам, возможность с...",nan,nan,Полная занятость,False,...,False,False,False,False,False,False,False,"БУЗ ВО ""ВОЛОГОДСКИЙ ОБЛАСТНОЙ НАРКОЛОГИЧЕСКИЙ ...",nan,nan
1,fafe2e23-0e2c-11e5-9cc4-3bdbd1a6e39d,nan,False,nan,NaN,NaN,nan,nan,Полная занятость,False,...,False,False,False,False,False,False,False,"ООО ""МЦ ""ПАНАЦЕЯ""",nan,nan
2,7eb9edc1-fd76-11e4-8a5b-3bdbd1a6e39d,nan,False,nan,NaN,"<p>&nbsp;</p> <p>Фельдшер отделения СМП, Ковжи...",nan,nan,Полная занятость,False,...,False,False,False,False,False,False,False,"БУЗ ВО ""ВЫТЕГОРСКАЯ ЦРБ""",nan,nan
3,2e68ddc3-0499-11e5-b3df-3bdbd1a6e39d,nan,False,nan,NaN,NaN,nan,nan,Полная занятость,False,...,False,False,False,False,False,False,False,"БПОУ ВО ""БОРИСОГЛЕБСКМЕДКОЛЛЕДЖ""",nan,nan
4,8460a3b2-fd7e-11e4-8a5b-3bdbd1a6e39d,nan,False,nan,NaN,NaN,nan,nan,Полная занятость,False,...,False,False,False,False,False,False,False,"БУЗ ВО ""ЛИСКИНСКАЯ РБ""",nan,nan


### and now something that actualy works

In [51]:
# from sklearn import ensemble
# from sklearn.inspection import permutation_importance
# from sklearn.metrics import mean_squared_error
# from sklearn.model_selection import train_test_split

# params = {
#     "n_estimators": 500,
#     "max_depth": 4,
#     "min_samples_split": 5,
#     "learning_rate": 0.01,
#     "loss": "squared_error",
# }

# reg = ensemble.GradientBoostingRegressor(**params)
# reg.fit(train_data, train_labels)

# mse = mean_squared_error(y_test, reg.predict(X_test))
# print("The mean squared error (MSE) on test set: {:.4f}".format(mse))

In [48]:
from numpy import shape
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import hstack, csr_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.preprocessing import OrdinalEncoder

In [59]:


# Handling missing values and converting text features to string type where necessary
# df_ts[text_features] = df_ts[text_features].fillna('не указано').astype(str)

for feature in text_features:
  df_ts[feature] = df_ts[feature].apply(lambda x: x if x==x else 'не указано')

# Apply TF-IDF transformation to text features
vectorizers = {feature: TfidfVectorizer(max_features=50) for feature in text_features}
X_text_features = [vectorizers[feature].fit_transform(df_ts[feature]) for feature in text_features]

# Combine all text features into a single sparse matrix
X_text = hstack(X_text_features)

# Handling numerical features
X_nums = df_ts[nums].fillna(0).astype('float32')  # Replace NaN with 0 if necessary


In [60]:
# Handling categorical features
ordinal_encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)
X_cats = ordinal_encoder.fit_transform(df_ts[cats].fillna('не указано'))  # Replace NaN with a placeholder if necessary

# Combine all features into a single matrix
X_combined = hstack([X_text, csr_matrix(X_nums), csr_matrix(X_cats)])

# Define the target variable y_salary
y_salary = df_ts['salary'].astype('float32')  # Ensure 'salary' is the correct column name

# Scale the combined features (excluding text features that are already scaled)
scaler = StandardScaler(with_mean=False)
X_combined = scaler.fit_transform(X_combined)

# Splitting the data into training and testing sets
X_train_sal, X_test_sal, y_train_sal, y_test_sal = train_test_split(X_combined, y_salary, test_size=0.2, random_state=42)

# Converting to dense format for the salary prediction model
X_train_sal = X_train_sal.toarray()
X_test_sal = X_test_sal.toarray()

In [61]:

# Improved model for salary prediction using Gradient Boosting
model_salary = GradientBoostingRegressor(
    n_estimators=100,
    learning_rate=0.1,
    max_depth=5,
    random_state=42,
    loss='squared_error',
    verbose = 1
)

# Training the salary model
model_salary.fit(X_train_sal, y_train_sal)

# Predictions on the test set
y_pred_sal = model_salary.predict(X_test_sal)


      Iter       Train Loss   Remaining Time 
         1   492790846.8469           23.33m
         2   472367362.1208           21.35m
         3   453194887.5420           20.41m
         4   436397490.3770           20.14m
         5   423782366.5139           19.79m
         6   411695659.5831           19.49m
         7   401027009.0281           19.49m
         8   391993141.7528           19.24m
         9   383964978.1366           19.07m
        10   376290370.2063           18.79m
        20   329651458.8325           16.47m
        30   303881650.2924           14.32m
        40   288466721.8954           12.26m
        50   277360528.6918           10.20m
        60   269297306.0523            8.19m
        70   263057866.0823            6.14m
        80   257913312.8590            4.10m
        90   253142628.5452            2.05m
       100   248244678.4358            0.00s


In [62]:
path = "https://lodmedia.hb.bizmrg.com/case_files/1125024/test_dataset_vprod_test.zip"
zt = zipfile.ZipFile(io.BytesIO(urlopen(path).read()))

In [63]:
ts=zt.open("TEST_SAL.csv")
# df_test = read_csv(ts)
df_test = read_csv(ts)

del zt # чтобы влезло /

In [64]:
# убираем фичи, для которых количесвто уникальных значений == 1
for i in df_test.columns:
  if len(df_test[i].unique())==1:
    df_test = df_test.drop(i, axis=1)


In [65]:
for feature in social_protected_ids:
  df_test[feature] = feature in df_test["social_protected_ids"]
for feature in vacancy_benefit_ids:
  df_test[feature] = feature in df_test["vacancy_benefit_ids"]

In [66]:
for cat in driver_cats:
  df_test[f"driver_cat_{cat}"] = df_test["required_drive_license"].apply(lambda x: "True" if cat in x else "False")
  cats.append(f"driver_cat_{cat}")

try:
  text_features.remove("required_drive_license")
except:
  pass


In [67]:
df_test = df_test.drop("required_drive_license", axis=1)

In [68]:
df_test["company_json_name"] = df_test["company"].apply(lambda x: loads(x)["name"]
                                                    if "name" in loads(x)
                                                    else "Название не указано") #



In [69]:
df_test.keys()

Index(['id', 'accommodation_capability', 'accommodation_type',
       'additional_premium', 'additional_requirements', 'bonus_type',
       'measure_type', 'busy_type', 'career_perspective', 'change_time',
       'code_external_system', 'code_profession', 'code_professional_sphere',
       'company_code', 'contact_person', 'data_ids', 'date_create',
       'date_modify', 'education', 'education_speciality', 'metro_ids',
       'is_mobility_program', 'is_quoted', 'need_medcard', 'okso_code',
       'original_source_type', 'other_vacancy_benefit',
       'position_requirements', 'position_responsibilities', 'published_date',
       'regionName', 'company_business_size', 'required_certificates',
       'required_experience', 'retraining_capability', 'retraining_grant',
       'schedule_type', 'social_protected_ids', 'source_type',
       'state_region_code', 'status', 'transport_compensation',
       'vacancy_address_additional_info', 'vacancy_address',
       'vacancy_address_code', 'vac

In [70]:
# keys_lg = ['code_language', 'id_owner', 'level', 'type']

In [71]:
df_test["LK_code_language"] = df_test["languageKnowledge"].apply(lambda x: loads(x)[0]["code_language"] if x!="[]" else "nan")

KeyError: 'languageKnowledge'

In [ ]:
df_ts["LK_level"] = df_ts["languageKnowledge"].apply(lambda x: (loads(x)[0]["level"] if "level" in loads(x)[0] else "Не указан") if x!="[]" else "nan")

cats.append("LK_code_language")
cats.append("LK_level")

KeyError: 'languageKnowledge'

In [72]:
df_test["hardSkills"] = df_test["hardSkills"].apply(lambda x: loads(x)[0]["hard_skill_name"] if x!="[]" else "nan")

In [73]:
df_test["softSkills"] = df_test["softSkills"].apply(lambda x: loads(x)[0]["soft_skill_name"] if x!="[]" else "nan")

In [74]:
df_test = df_test.drop("company",axis=1)

In [75]:
df_test = df_test.drop("languageKnowledge",axis=1)

KeyError: "['languageKnowledge'] not found in axis"

In [76]:
ids = df_test['id']


In [77]:


# Handling missing values and converting text features to string type where necessary
df_test[text_features] = df_test[text_features].fillna('не указано').astype(str)

# Apply TF-IDF transformation to text features
# vectorizers = {feature: TfidfVectorizer(max_features=50) for feature in text_features}
X_text_features = [vectorizers[feature].transform(df_test[feature]) for feature in text_features]

# Combine all text features into a single sparse matrix
X_text = hstack(X_text_features)

# Handling numerical features
X_nums = df_test[nums].fillna(0).astype('float32')  # Replace NaN with 0 if necessary

# Handling categorical features
# ordinal_encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)
X_cats = ordinal_encoder.transform(df_test[cats].fillna('не указано'))  # Replace NaN with a placeholder if necessary

# Combine all features into a single matrix
X_combined = hstack([X_text, csr_matrix(X_nums), csr_matrix(X_cats)])

# Define the target variable y_salary
y_salary = df_test['salary'].astype('float32')  # Ensure 'salary' is the correct column name

# Scale the combined features (excluding text features that are already scaled)
# scaler = StandardScaler(with_mean=False)
X_combined = scaler.transform(X_combined)

# Splitting the data into training and testing sets
X_test_sal = X_combined

# Converting to dense format for the salary prediction model
X_test_sal = X_test_sal.toarray()

KeyError: "['academic_degree', 'contact_source', 'is_uzbekistan_recruitment'] not in index"

In [ ]:
answers = model_salary.predict(X_test_sal)

In [ ]:
y_pred_sal

from numpy import nan
answers = DataFrame(y_pred_sal)
answers["id"] = y_pred_sal
answers["job_title"] = nan
answers["task_type"] = answers["job_title"].apply(lambda x: "SAL")

answers.head(10)

,0,id,job_title,task_type
0,34999.846640,34999.846640,NaN,SAL
1,19243.141949,19243.141949,NaN,SAL
2,104948.937853,104948.937853,NaN,SAL
3,35992.537154,35992.537154,NaN,SAL
4,23974.921633,23974.921633,NaN,SAL
5,27998.145641,27998.145641,NaN,SAL
6,30000.197579,30000.197579,NaN,SAL
7,33000.069147,33000.069147,NaN,SAL
8,19243.141949,19243.141949,NaN,SAL
9,33989.708196,33989.708196,NaN,SAL


In [ ]:
# Evaluating model performance
mae = mean_absolute_error(y_test_sal, y_pred_sal)
rmse = mean_squared_error(y_test_sal, y_pred_sal, squared = False)
print(f"Mean Absolute Error on test set: {mae}")
print(f"Root Mean Squared Error on test set: {rmse}")

print(f"target metric: {0.1-rmse/33000}")

Mean Absolute Error on test set: 12.086686280046433
Root Mean Squared Error on test set: 50.929235589600076
target metric: 0.09845668983061819


In [ ]:
# Creating examples for submission
df_TEST_SAL_example = df_ts.iloc[:len(y_pred_sal)]

def create_submission_part(test_df, name_of_predict_column, predictions):
    submission = pd.DataFrame([])
    submission['id'] = test_df['id']
    submission[name_of_predict_column] = predictions
    return submission

submission_SAL_part = create_submission_part(df_TEST_SAL_example, 'salary', y_pred_sal.flatten())

# Output results
print(submission_SAL_part.head())

#Test

In [ ]:
path = "https://lodmedia.hb.bizmrg.com/case_files/1125024/test_dataset_vprod_test.zip"
zt = zipfile.ZipFile(io.BytesIO(urlopen(path).read()))

In [ ]:
ts=zt.open("TEST_SAL.csv")
# df_test = read_csv(ts)
df_test = read_csv(ts)

del zt # чтобы влезло /

In [ ]:
# убираем фичи, для которых количесвто уникальных значений == 1
for i in df_test.columns:
  if len(df_test[i].unique())==1:
    df_test = df_test.drop(i, axis=1)


In [ ]:
for feature in social_protected_ids:
  df_test[feature] = feature in df_test["social_protected_ids"]
for feature in vacancy_benefit_ids:
  df_test[feature] = feature in df_test["vacancy_benefit_ids"]

In [ ]:
for cat in driver_cats:
  df_test[f"driver_cat_{cat}"] = df_test["required_drive_license"].apply(lambda x: "True" if cat in x else "False")
  cats.append(f"driver_cat_{cat}")

try:
  text_features.remove("required_drive_license")
except:
  pass


In [ ]:
df_test = df_test.drop("required_drive_license", axis=1)

In [ ]:
df_test["company_json_name"] = df_test["company"].apply(lambda x: loads(x)["name"]
                                                    if "name" in loads(x)
                                                    else "Название не указано") #



In [ ]:
df_test.keys()

Index(['id', 'accommodation_capability', 'accommodation_type',
       'additional_premium', 'additional_requirements', 'bonus_type',
       'measure_type', 'busy_type', 'career_perspective', 'change_time',
       'code_external_system', 'code_profession', 'code_professional_sphere',
       'company_code', 'contact_person', 'data_ids', 'date_create',
       'date_modify', 'education', 'education_speciality', 'metro_ids',
       'is_mobility_program', 'is_quoted', 'need_medcard', 'okso_code',
       'original_source_type', 'other_vacancy_benefit',
       'position_requirements', 'position_responsibilities', 'published_date',
       'regionName', 'company_business_size', 'required_certificates',
       'required_experience', 'retraining_capability', 'retraining_grant',
       'schedule_type', 'social_protected_ids', 'source_type',
       'state_region_code', 'status', 'transport_compensation',
       'vacancy_address_additional_info', 'vacancy_address',
       'vacancy_address_code', 'vac

In [ ]:
# keys_lg = ['code_language', 'id_owner', 'level', 'type']

In [ ]:
df_test["LK_code_language"] = df_test["languageKnowledge"].apply(lambda x: loads(x)[0]["code_language"] if x!="[]" else "nan")

KeyError: 'languageKnowledge'

In [ ]:
df_ts["LK_level"] = df_ts["languageKnowledge"].apply(lambda x: (loads(x)[0]["level"] if "level" in loads(x)[0] else "Не указан") if x!="[]" else "nan")

cats.append("LK_code_language")
cats.append("LK_level")

KeyError: 'languageKnowledge'

In [ ]:
df_test["hardSkills"] = df_test["hardSkills"].apply(lambda x: loads(x)[0]["hard_skill_name"] if x!="[]" else "nan")

In [ ]:
df_test["softSkills"] = df_test["softSkills"].apply(lambda x: loads(x)[0]["soft_skill_name"] if x!="[]" else "nan")

In [ ]:
df_test = df_test.drop("company",axis=1)

In [ ]:
df_test = df_test.drop("languageKnowledge",axis=1)

KeyError: "['languageKnowledge'] not found in axis"

In [ ]:
ids = df_test['id']


In [ ]:


# Handling missing values and converting text features to string type where necessary
df_test[text_features] = df_test[text_features].fillna('не указано').astype(str)

# Apply TF-IDF transformation to text features
# vectorizers = {feature: TfidfVectorizer(max_features=50) for feature in text_features}
X_text_features = [vectorizers[feature].transform(df_test[feature]) for feature in text_features]

# Combine all text features into a single sparse matrix
X_text = hstack(X_text_features)

# Handling numerical features
X_nums = df_test[nums].fillna(0).astype('float32')  # Replace NaN with 0 if necessary

# Handling categorical features
# ordinal_encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)
X_cats = ordinal_encoder.transform(df_test[cats].fillna('не указано'))  # Replace NaN with a placeholder if necessary

# Combine all features into a single matrix
X_combined = hstack([X_text, csr_matrix(X_nums), csr_matrix(X_cats)])

# Define the target variable y_salary
y_salary = df_test['salary'].astype('float32')  # Ensure 'salary' is the correct column name

# Scale the combined features (excluding text features that are already scaled)
# scaler = StandardScaler(with_mean=False)
X_combined = scaler.transform(X_combined)

# Splitting the data into training and testing sets
X_test_sal = X_combined

# Converting to dense format for the salary prediction model
X_test_sal = X_test_sal.toarray()

KeyError: "['retraining_grant_value', 'salary', 'salary_min', 'salary_max'] not in index"

In [ ]:
answers = model_salary.predict(X_test_sal)

In [ ]:
from numpy import nan
answers = DataFrame(answers)
answers["id"] = ids
answers["job_title"] = nan
answers["task_type"] = answers["job_title"].apply(lambda x: "SAL")

#Script

Скрипт автоматической проверки

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

In [ ]:
pred_submission = pd.read_csv('submission.csv')
true_submission = pd.read_csv('grounded_true.csv')

In [ ]:
pred_submission

,id,job_title,task_type,salary
0,52860148-26da-11e8-a184-9122a281f90e,специалист,RES,NaN
1,e3c857b0-26a0-11e7-bcca-736ab11edb0c,специалист,RES,NaN
2,d09f8cf0-2b52-11e8-a7fa-ef76bd2a03c1,специалист,RES,NaN
3,7f79c860-2920-11e8-9025-ef76bd2a03c1,специалист,RES,NaN
4,13af7f60-2b1b-11e8-b956-037acc02728d,специалист,RES,NaN
...,...,...,...,...
195,a902d967-35e4-11e5-a8c4-239645b044d5,NaN,SAL,32000.0
196,a5532597-35e9-11e5-a8c4-239645b044d5,NaN,SAL,32000.0
197,8749da83-3661-11e5-a0ef-3bdbd1a6e39d,NaN,SAL,32000.0
198,18649bd6-366d-11e5-a8c4-239645b044d5,NaN,SAL,32000.0


In [ ]:
true_submission

,id,job_title,task_type,salary
0,52860148-26da-11e8-a184-9122a281f90e,инспектор,RES,NaN
1,e3c857b0-26a0-11e7-bcca-736ab11edb0c,геолог,RES,NaN
2,d09f8cf0-2b52-11e8-a7fa-ef76bd2a03c1,геолог,RES,NaN
3,7f79c860-2920-11e8-9025-ef76bd2a03c1,фрезеровщик,RES,NaN
4,13af7f60-2b1b-11e8-b956-037acc02728d,геолог,RES,NaN
...,...,...,...,...
195,a902d967-35e4-11e5-a8c4-239645b044d5,NaN,SAL,19242.0
196,a5532597-35e9-11e5-a8c4-239645b044d5,NaN,SAL,40000.0
197,8749da83-3661-11e5-a0ef-3bdbd1a6e39d,NaN,SAL,34000.0
198,18649bd6-366d-11e5-a8c4-239645b044d5,NaN,SAL,32000.0


In [ ]:
N = 100 #только в этом примере
if len(pred_submission)!=N*2:
  raise Exception('Необходимо сделать предсказание для всех строк из тестового датасета, суммарная длина получившегося сабмита должна быть N + N = 2N ')

if len(set(pred_submission['id'].unique()) & set(true_submission['id'].unique()))!=N*2:
  raise Exception('Столбец id в submission должен иметь те же значения, что и столбец id в grounded true')

In [ ]:
try:
    df = true_submission.merge(pred_submission, how = 'left', on = 'id', suffixes=('_true','_pred'))
except Exception:
    assert False, "Ошибка при сравнении grounded_true и submission"

In [ ]:
df

,id,job_title_true,task_type_true,salary_true,job_title_pred,task_type_pred,salary_pred
0,52860148-26da-11e8-a184-9122a281f90e,инспектор,RES,NaN,специалист,RES,NaN
1,e3c857b0-26a0-11e7-bcca-736ab11edb0c,геолог,RES,NaN,специалист,RES,NaN
2,d09f8cf0-2b52-11e8-a7fa-ef76bd2a03c1,геолог,RES,NaN,специалист,RES,NaN
3,7f79c860-2920-11e8-9025-ef76bd2a03c1,фрезеровщик,RES,NaN,специалист,RES,NaN
4,13af7f60-2b1b-11e8-b956-037acc02728d,геолог,RES,NaN,специалист,RES,NaN
...,...,...,...,...,...,...,...
195,a902d967-35e4-11e5-a8c4-239645b044d5,NaN,SAL,19242.0,NaN,SAL,32000.0
196,a5532597-35e9-11e5-a8c4-239645b044d5,NaN,SAL,40000.0,NaN,SAL,32000.0
197,8749da83-3661-11e5-a0ef-3bdbd1a6e39d,NaN,SAL,34000.0,NaN,SAL,32000.0
198,18649bd6-366d-11e5-a8c4-239645b044d5,NaN,SAL,32000.0,NaN,SAL,32000.0


In [ ]:
if sum(df['task_type_true'] != df['task_type_pred']):
  raise Exception('У вас должно получиться N строк с задачей RES и N строк с задачей SAL')

In [ ]:
RES_part = df[df['task_type_true']=='RES']
SAL_part = df[df['task_type_true']=='SAL']

In [ ]:
if sum(RES_part['job_title_pred'].isna()):
  raise Exception('Замените все NaN значения в столбце "job_title"')

In [ ]:
RES_part = RES_part.astype({'job_title_pred':'string'})

if not RES_part['job_title_pred'].dtype == 'string':
 raise Exception('Столбец "job_title" должен иметь тип данных "string"')
if not SAL_part['salary_pred'].dtype in ['float64','float32', 'int64','int32']:
  raise Exception('Столбец "salary_pred" должен иметь числовой тип данных')

In [ ]:
RES_part

,id,job_title_true,task_type_true,salary_true,job_title_pred,task_type_pred,salary_pred
0,52860148-26da-11e8-a184-9122a281f90e,инспектор,RES,NaN,специалист,RES,NaN
1,e3c857b0-26a0-11e7-bcca-736ab11edb0c,геолог,RES,NaN,специалист,RES,NaN
2,d09f8cf0-2b52-11e8-a7fa-ef76bd2a03c1,геолог,RES,NaN,специалист,RES,NaN
3,7f79c860-2920-11e8-9025-ef76bd2a03c1,фрезеровщик,RES,NaN,специалист,RES,NaN
4,13af7f60-2b1b-11e8-b956-037acc02728d,геолог,RES,NaN,специалист,RES,NaN
...,...,...,...,...,...,...,...
95,ce8046d0-2ebd-11e8-b956-037acc02728d,почтальон,RES,NaN,специалист,RES,NaN
96,62eca040-2fde-11e8-8a6b-ef76bd2a03c1,специалист,RES,NaN,специалист,RES,NaN
97,0e16fc40-2fdf-11e8-8a6b-ef76bd2a03c1,специалист,RES,NaN,специалист,RES,NaN
98,70dca140-2fdf-11e8-8a6b-ef76bd2a03c1,специалист,RES,NaN,специалист,RES,NaN


In [ ]:
#RES
f1 = f1_score(RES_part['job_title_pred'], RES_part['job_title_true'], average = 'micro')
#SAL
rmse = mean_squared_error(SAL_part['salary_pred'], SAL_part['salary_true'], squared=False)
custom_metric = max(0,1-(rmse/33000))

final_metric = f1 + custom_metric

In [ ]:
#макс 2.0
final_metric

0.1840777768795854